# ICAPS24 SkDecide Tutorial: solving PDDL problems with classical planning, and reinforcement learning solvers

In [ ]:
!wget https://raw.githubusercontent.com/fteicht/icaps24-skdecide-tutorial/main/notebooks/install_skdecide.py

from install_skdecide import install_skdecide
install_skdecide(using_nightly_version=False, force_reinstall=False)

![Solver code generator](pictures/skdecide_solver_generator.png)

In [ ]:
from typing import Any

from skdecide.core import TransitionOutcome, Value, Space
from skdecide.domains import RLDomain
from skdecide.builders.domain import UnrestrictedActions, FullyObservable, Renderable
from skdecide.hub.space.gym import DictSpace, GymSpace, MultiDiscreteSpace
from skdecide.utils import rollout

import pyRDDLGym
from pyRDDLGym_rl.core.env import SimplifiedActionRDDLEnv

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

class D(RLDomain, UnrestrictedActions, FullyObservable, Renderable):
    T_state = DictSpace  # Type of states
    T_observation = T_state  # Type of observations
    T_event = MultiDiscreteSpace  # Type of events
    T_value = float  # Type of transition values (rewards or costs)
    T_info = None  # Type of additional information in environment outcome


class RDDLDomainUnrestrictedActions(D):

    def __init__(self, rddl_domain: str, rddl_instance: str):
        self.rddl_gym_env = pyRDDLGym.make(rddl_domain, rddl_instance, base_class=SimplifiedActionRDDLEnv, enforce_action_constraints=True)

        plt.ioff()
        self.fig, self.ax = plt.subplots(1)
        self.ax.axis("off")
        plt.ion()
        self.img = None

    def _state_step(
        self, action: D.T_event
    ) -> TransitionOutcome[D.T_state, Value[D.T_value], D.T_predicate, D.T_info]:
        next_state, reward, terminated, truncated, _ = self.rddl_gym_env.step(action)
        return TransitionOutcome(
            state=next_state, value=Value(reward=reward), termination=terminated or truncated
        )

    def _get_action_space_(self) -> Space[D.T_event]:
        return GymSpace(self.rddl_gym_env.action_space)

    def _state_reset(self) -> D.T_state:
        # SkDecide only needs the state, not the info
        return self.rddl_gym_env.reset()[0]

    def _get_observation_space_(self) -> Space[D.T_observation]:
        return GymSpace(self.rddl_gym_env.observation_space)

    def _render_from(self, memory: D.T_state = None, **kwargs: Any) -> Any:
        # We do not want the image to be displayed in a pygame window, but rather in this notebook
        rddl_gym_img = self.rddl_gym_env.render(to_display=False)
        if self.img is None:
            self.img = self.ax.imshow(rddl_gym_img)
        else:
            self.img.set_data(rddl_gym_img)
        self.fig.canvas.draw()
        clear_output(wait=True)
        display(self.fig)

    def close(self):
        plt.close(self.fig)